In [214]:
import os
from io import StringIO
import tabula

# analysis
import numpy as np
import pandas as pd

# api
import json
import requests
import prettytable

# Get data

**What data do we need?**
- GDP by states by industry: https://www.bea.gov/system/files/2020-01/qgdpstate0120_2.pdf
- Employmnet by states by industry:
    - In order to get the data, we probably need to get the employment data by industry from each states' website.
        - NY: https://labor.ny.gov/stats/2016-2026-Statewide-and-Regional-Long-Term-Industry-Projections.xlsx
        - NJ: https://www.nj.gov/labor/lpa/employ/indoccpj/20162026%20Industry%20Employment%20Projections%20Reports.xlsx
        - ...so the 2016-2026 employment projection is avaliable on each states websites
        
- Labor Productivity by states by industry:
    - Private nonfarm productivity and costs by state and region: https://www.bls.gov/lpc/lpc-by-state-and-region.xlsx
    - https://www.bls.gov/opub/mlr/2019/article/bls-publishes-experimental-state-level-labor-productivity-measures.htm
        

## 1.0 GDP by states by industry
**API:** https://www.bls.gov/developers/api_signature_v2.htm

#### Try API

In [151]:
# my_bls_api = "ccdcbc9910a744a5be046d1aa70ac25c"

In [152]:
# headers = {'Content-type': 'application/json'}
# data = json.dumps({"seriesid": ['IPUAN1111__L010','IPUAN1111__L011'],"startyear":"2000", "endyear":"2019"})
# p = requests.post('https://api.bls.gov/ccdcbc9910a744a5be046d1aa70ac25c/v2/timeseries/data/', data=data, headers=headers)
# json_data = json.loads(p.text)
# for series in json_data['Results']['series']:
#     x=prettytable.PrettyTable(["series_id","year","period","value","footnotes"])
#     seriesId = series['seriesID']
#     for item in series['data']:
#         year = item['year']
#         period = item['period']
#         value = item['value']
#         footnotes=""
#         for footnote in item['footnotes']:
#             if footnote:
#                 footnotes = footnotes + footnote['text'] + ','
# #         if 'M01' <= period <= 'M12':
#         x.add_row([seriesId,year,period,value,footnotes[0:-1]])
#     output = open(seriesId + '.txt','w')
#     output.write (x.get_string())
#     output.close()

#### Reading from PDF


#### 1. Current-Dollar Gross Domestic Product (GDP) by State and Region, 2018:Q1-2019:Q3

In [157]:
gdp_df = tabula.read_pdf(os.path.join(os.getcwd(),"data/qgdpstate0120_2.pdf"), 
                         pages = 7)

In [158]:
gdp_df = gdp_df[0]

In [159]:
gdp_df = gdp_df.iloc[3:]

In [160]:
gdp_df = gdp_df.iloc[:,[0,1,2,3,4]]

In [161]:
gdp_df = pd.concat([gdp_df,gdp_df["Millions of dollars"].str.split(" ",expand = True)],axis=1)

In [162]:
gdp_df = gdp_df.drop(columns=["Millions of dollars","Unnamed: 3",5,6,7,8])

In [163]:
gdp_df.columns = ["areas","2018-1","2018-2","2018-3","2018-4","2019-1","2019-2","2019-3"]

In [164]:
gdp_df = gdp_df.reset_index(drop=True)

In [165]:
gdp_df.head()

,areas,2018-1,2018-2,2018-3,2018-4,2019-1,2019-2,2019-3
0,United States,"20,163,159","20,510,177","20,749,752","20,897,804","21,098,827","21,340,267","21,542,540"
1,New England,"1,070,863","1,084,947","1,096,174","1,101,531","1,118,703","1,130,084","1,142,586"
2,Connecticut,"272,854","273,884","278,295","277,874","281,659","284,357","287,560"
3,Maine,"63,698","64,746","65,434","65,545","66,590","67,138","67,905"
4,Massachusetts,"558,472","568,220","573,506","577,754","586,347","592,588","599,092"


#### 2. Contributions to Percent Change in Real Gross Domestic Product (GDP) by State and Region, 2019:Q2-2019:Q3--Table Ends

### Part 1

In [392]:
gdp_by_state_by_inds_p1 = tabula.read_pdf(os.path.join(os.getcwd(),"data/qgdpstate0120_2.pdf"), 
                         pages = 5)

In [393]:
gdp_by_state_by_inds_p1 = gdp_by_state_by_inds_p1[0]

In [394]:
gdp_by_state_by_inds_p1 = gdp_by_state_by_inds_p1.dropna(axis=0, how='all').dropna(axis=1, how='all')

In [395]:
gdp_by_state_by_inds_p1.head(6)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Seasonally adjusted at annual rates,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,NaN,NaN,NaN,NaN,NaN,NaN,Percentage points,NaN,NaN,NaN
1,NaN,Percent change,NaN,"Mining,",NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,in real GDP by,"Agriculture,","quarrying, and",NaN,NaN,Durable goods Nondurable goods,NaN,Transportation and,Finance and
3,NaN,state,"forestry, fishing insuranceand hunting",Informationoil and gas,Utilities,Construction,Wholesale trade manufacturing manufacturing,Retail trade,warehousing,NaN
4,NaN,NaN,NaN,extraction,NaN,NaN,NaN,NaN,NaN,NaN
5,United States,2.1,0.06,0.11,-0.26,-0.09,0.01 0.46 0.18,0.43,-0.03,0.29 -0.42


In [396]:
gdp_by_state_by_inds_p1.iloc[3]

Unnamed: 0                                                                     NaN
Unnamed: 1                                                                   state
Unnamed: 2                                  forestry, fishing insuranceand hunting
Unnamed: 3                                                  Informationoil and gas
Unnamed: 4                                                               Utilities
Unnamed: 5                                                            Construction
Seasonally adjusted at annual rates    Wholesale trade manufacturing manufacturing
Unnamed: 6                                                            Retail trade
Unnamed: 7                                                             warehousing
Unnamed: 8                                                                     NaN
Name: 3, dtype: object

In [397]:
gdp_by_state_by_inds_p1 = gdp_by_state_by_inds_p1.loc[5:]

In [398]:
gdp_by_state_by_inds_p1 = pd.concat([gdp_by_state_by_inds_p1,gdp_by_state_by_inds_p1[
                                            'Seasonally adjusted at annual rates'].str.split(" ", expand=True)], axis=1)

In [399]:
gdp_by_state_by_inds_p1 = gdp_by_state_by_inds_p1.drop(columns=['Seasonally adjusted at annual rates'])

In [404]:
gdp_by_state_by_inds_p1 = pd.concat([gdp_by_state_by_inds_p1,gdp_by_state_by_inds_p1[
                                            'Unnamed: 8'].str.split(" ", expand=True)], axis=1)

In [406]:
gdp_by_state_by_inds_p1 = gdp_by_state_by_inds_p1.drop(columns=['Unnamed: 8'])

In [407]:
gdp_by_state_by_inds_p1.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,0,1,2,0,1
5,United States,2.1,0.06,0.11,-0.26,-0.09,0.43,-0.03,0.01,0.46,0.18,0.29,-0.42
6,New England,2.1,0.01,0.01,-0.24,-0.09,0.39,-0.02,0.02,0.33,0.19,0.34,-0.47
7,Connecticut,2.1,0.00,0.00,-0.27,-0.15,0.39,-0.01,0.02,0.71,0.30,0.36,-0.55
8,Maine,2.1,0.01,0.00,-0.22,0.04,0.72,0.01,-0.02,0.29,0.24,0.17,-0.43
9,Massachusetts,2.2,0.02,0.01,-0.21,-0.11,0.31,-0.03,0.04,0.21,0.12,0.40,-0.43


In [408]:
cols = ["areas", "states_overall","Agriculture,forestry, fishing insuranceand hunting", 
        "Mining,quarrying, and oil and gas extraction","Utilities","Construction", 
        "Retail trade","Transportation and warehousing","Durable goods manufacturing",
        "Nondurable goods manufacturing", "Wholesale trade", "Information", 
        "Finance and insurance"]

In [409]:
gdp_by_state_by_inds_p1.columns = cols

In [420]:
gdp_by_state_by_inds_p1 = gdp_by_state_by_inds_p1.reset_index(drop=True)

### Part 2

In [345]:
gdp_by_state_by_inds_p2 = tabula.read_pdf(os.path.join(os.getcwd(),"data/qgdpstate0120_2.pdf"), 
                         pages = 6)

In [346]:
gdp_by_state_by_inds_p2 = gdp_by_state_by_inds_p2[0]

In [347]:
gdp_by_state_by_inds_p2.iloc[:5,:5]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,NaN,NaN,NaN,NaN,NaN
1,NaN,Other services (except Real estate and governm...,Management of companies and enterprises,NaN,NaN
2,United States,0.11 0.42,0.16,NaN,NaN
3,New England,0.09 0.56,0.20,NaN,NaN
4,Connecticut,0.03 0.37,0.19,NaN,NaN


In [348]:
gdp_by_state_by_inds_p2 = gdp_by_state_by_inds_p2[2:]

In [351]:
gdp_by_state_by_inds_p2[["Real estate and rental and leasing",
                      "Professional, scientific, and technical services"]] = gdp_by_state_by_inds_p2["Unnamed: 1"].str.split(" ",expand = True)

In [352]:
gdp_by_state_by_inds_p2[["Educational services",
                      "Health care and social assistance"]] = gdp_by_state_by_inds_p2[
                                                            "Seasonally adjusted at annual rates"
                                                                                  ].str.split(" ", expand = True)

In [353]:
gdp_by_state_by_inds_p2[["Other services (except government and government enterprises)",
                      "Government and government enterprises"]] = gdp_by_state_by_inds_p2["Unnamed: 14"].str.split(" ", expand = True)

In [354]:
gdp_by_state_by_inds_p2 = gdp_by_state_by_inds_p2.drop(columns= ["Unnamed: 1","Unnamed: 14","Seasonally adjusted at annual rates"])

In [355]:
gdp_by_state_by_inds_p2 = gdp_by_state_by_inds_p2.dropna(axis=1)

In [356]:
gdp_by_state_by_inds_p2.iloc[:5,:5]

,Unnamed: 0,Unnamed: 2,Unnamed: 5,Unnamed: 9,Unnamed: 11
2,United States,0.16,0.10,0.02,0.17
3,New England,0.20,0.12,0.04,0.17
4,Connecticut,0.19,0.11,0.04,0.11
5,Maine,0.27,0.15,0.05,0.33
6,Massachusetts,0.20,0.10,0.03,0.16


In [357]:
name_dic = {"Unnamed: 0":"areas",
            "Unnamed: 2":"Management of companies and enterprises",
            "Unnamed: 5":"Administrative and support and waste management and remediation services",
            "Unnamed: 9":"Arts, entertainment, and recreation", 
            "Unnamed: 11":"Accomodation and food services"}

In [358]:
gdp_by_state_by_inds_p2 = gdp_by_state_by_inds_p2.rename(columns=name_dic)

In [418]:
gdp_by_state_by_inds_p2 = gdp_by_state_by_inds_p2.reset_index(drop=True)

In [419]:
gdp_by_state_by_inds_p2.dtypes

areas                                                                        object
management_of_companies_and_enterprises                                      object
administrative_and_support_and_waste_management_and_remediation_services     object
arts_entertainment_and_recreation                                           float64
accomodation_and_food_services                                               object
educational_services                                                         object
health_care_and_social_assistance                                            object
real_estate_and_rental_and_leasing                                           object
professional_scientific_and_technical_services                               object
other_services_except_government_and_government_enterprises_                 object
government_and_government_enterprises                                        object
dtype: object

### Join part one and part two

In [421]:
gdp_by_state_by_inds = pd.concat([gdp_by_state_by_inds_p1,gdp_by_state_by_inds_p2], axis = 1,)

In [422]:
gdp_by_state_by_inds.columns = gdp_by_state_by_inds.columns.str.replace(
                                                "\)|\(|,| ","_").str.replace("__","_").str.lower()

In [428]:
gdp_by_state_by_inds = gdp_by_state_by_inds.iloc[:,~gdp_by_state_by_inds.columns.duplicated()]

In [429]:
gdp_by_state_by_inds.head()

,areas,states_overall,agriculture_forestry_fishing_insuranceand_hunting,mining_quarrying_and_oil_and_gas_extraction,utilities,construction,retail_trade,transportation_and_warehousing,durable_goods_manufacturing,nondurable_goods_manufacturing,...,management_of_companies_and_enterprises,administrative_and_support_and_waste_management_and_remediation_services,arts_entertainment_and_recreation,accomodation_and_food_services,educational_services,health_care_and_social_assistance,real_estate_and_rental_and_leasing,professional_scientific_and_technical_services,other_services_except_government_and_government_enterprises_,government_and_government_enterprises
0,United States,2.1,0.06,0.11,-0.26,-0.09,0.43,-0.03,0.01,0.46,...,0.16,0.10,0.02,0.17,0.08,0.19,0.11,0.42,0.05,0.01
1,New England,2.1,0.01,0.01,-0.24,-0.09,0.39,-0.02,0.02,0.33,...,0.20,0.12,0.04,0.17,0.23,0.21,0.09,0.56,0.05,-0.01
2,Connecticut,2.1,0.00,0.00,-0.27,-0.15,0.39,-0.01,0.02,0.71,...,0.19,0.11,0.04,0.11,0.27,0.17,0.03,0.37,0.07,-0.04
3,Maine,2.1,0.01,0.00,-0.22,0.04,0.72,0.01,-0.02,0.29,...,0.27,0.15,0.05,0.33,0.06,0.15,0.07,0.26,0.05,-0.10
4,Massachusetts,2.2,0.02,0.01,-0.21,-0.11,0.31,-0.03,0.04,0.21,...,0.20,0.10,0.03,0.16,0.24,0.22,0.13,0.72,0.05,0.00


In [437]:
gdp_by_state_by_inds[]

,areas,states_overall,agriculture_forestry_fishing_insuranceand_hunting,mining_quarrying_and_oil_and_gas_extraction,utilities,construction,retail_trade,transportation_and_warehousing,durable_goods_manufacturing,nondurable_goods_manufacturing,...,management_of_companies_and_enterprises,administrative_and_support_and_waste_management_and_remediation_services,arts_entertainment_and_recreation,accomodation_and_food_services,educational_services,health_care_and_social_assistance,real_estate_and_rental_and_leasing,professional_scientific_and_technical_services,other_services_except_government_and_government_enterprises_,government_and_government_enterprises
2,Connecticut,2.1,0.00,0.00,-0.27,-0.15,0.39,-0.01,0.02,0.71,...,0.19,0.11,0.04,0.11,0.27,0.17,0.03,0.37,0.07,-0.04
3,Maine,2.1,0.01,0.00,-0.22,0.04,0.72,0.01,-0.02,0.29,...,0.27,0.15,0.05,0.33,0.06,0.15,0.07,0.26,0.05,-0.10
4,Massachusetts,2.2,0.02,0.01,-0.21,-0.11,0.31,-0.03,0.04,0.21,...,0.20,0.10,0.03,0.16,0.24,0.22,0.13,0.72,0.05,0.00
5,New Hampshire,2.2,0.00,0.01,-0.24,0.09,0.53,-0.07,0.07,0.08,...,0.12,0.20,0.04,0.12,0.15,0.22,0.08,0.57,0.04,0.06
6,Rhode Island,1.6,0.02,0.00,-0.27,-0.10,0.39,0.01,-0.11,0.16,...,0.27,0.22,0.06,0.28,0.37,0.35,0.05,0.28,0.04,-0.03
7,Vermont,1.8,-0.09,0.04,-0.33,-0.07,0.58,-0.02,0.00,0.20,...,0.09,0.12,0.03,0.29,0.16,0.22,0.07,0.41,0.05,-0.02
8,Mideast,1.2,0.00,0.05,-0.25,-0.14,0.35,0.00,-0.06,0.41,...,0.15,0.00,0.02,0.19,0.06,0.21,0.09,0.46,0.04,-0.07
9,Delaware,0.0,-0.07,0.00,-0.24,-0.11,0.31,-0.05,0.02,0.39,...,0.15,0.05,0.00,0.13,0.04,0.21,0.10,0.35,0.03,0.05
10,District of Columbia,1.4,0.00,0.00,-0.17,-0.07,0.11,-0.04,0.00,0.01,...,0.05,0.23,0.02,0.28,0.28,0.05,0.04,0.75,0.09,-0.42
11,Maryland,1.0,0.01,0.01,-0.34,-0.12,0.35,-0.03,-0.02,0.18,...,0.09,-0.01,0.04,0.17,0.08,0.19,0.10,0.52,0.05,-0.22


In [331]:
def get_gdp_by_states(data_dir):
    gdp_df = tabula.read_pdf(data_dir, pages = 7)
    gdp_df = gdp_df[0].iloc[3:,[0,1,2,3,4]]
    gdp_df = pd.concat([gdp_df,gdp_df["Millions of dollars"].str.split(" ",expand = True)],axis=1)
    gdp_df = gdp_df.drop(columns=["Millions of dollars","Unnamed: 3",5,6,7,8])
    gdp_df.columns = ["areas","2018-1","2018-2","2018-3","2018-4","2019-1","2019-2","2019-3"]
    gdp_df = gdp_df.reset_index(drop=True)
    
    return gdp_df

In [332]:
data_dir = os.path.join(os.getcwd(),"data/qgdpstate0120_2.pdf")

In [336]:
get_gdp_by_states(data_dir=data_dir).head()

,areas,2018-1,2018-2,2018-3,2018-4,2019-1,2019-2,2019-3
0,United States,"20,163,159","20,510,177","20,749,752","20,897,804","21,098,827","21,340,267","21,542,540"
1,New England,"1,070,863","1,084,947","1,096,174","1,101,531","1,118,703","1,130,084","1,142,586"
2,Connecticut,"272,854","273,884","278,295","277,874","281,659","284,357","287,560"
3,Maine,"63,698","64,746","65,434","65,545","66,590","67,138","67,905"
4,Massachusetts,"558,472","568,220","573,506","577,754","586,347","592,588","599,092"


In [430]:
def get_gdp_pct_by_states_by_inds(data_dir):
    
    # part 1
    gdp_by_state_by_inds_p1 = tabula.read_pdf(data_dir,pages = 5)
    gdp_by_state_by_inds_p1 = gdp_by_state_by_inds_p1[0]
    gdp_by_state_by_inds_p1 = gdp_by_state_by_inds_p1.dropna(axis=0, how='all').dropna(axis=1, how='all')
    gdp_by_state_by_inds_p1 = gdp_by_state_by_inds_p1.loc[5:]
    gdp_by_state_by_inds_p1 = pd.concat([gdp_by_state_by_inds_p1,gdp_by_state_by_inds_p1[
                                            'Seasonally adjusted at annual rates'].str.split(" ", expand=True)], axis=1)
    gdp_by_state_by_inds_p1 = gdp_by_state_by_inds_p1.drop(columns=['Seasonally adjusted at annual rates'])
    gdp_by_state_by_inds_p1 = pd.concat([gdp_by_state_by_inds_p1,gdp_by_state_by_inds_p1[
                                            'Unnamed: 8'].str.split(" ", expand=True)], axis=1)
    gdp_by_state_by_inds_p1 = gdp_by_state_by_inds_p1.drop(columns=['Unnamed: 8'])
    cols = ["areas", "states_overall","Agriculture,forestry, fishing insuranceand hunting", 
            "Mining,quarrying, and oil and gas extraction","Utilities","Construction", 
            "Retail trade","Transportation and warehousing","Durable goods manufacturing",
            "Nondurable goods manufacturing", "Wholesale trade", "Information", 
            "Finance and insurance"]
    gdp_by_state_by_inds_p1.columns = cols
    gdp_by_state_by_inds_p1 = gdp_by_state_by_inds_p1.reset_index(drop=True)

    #part 2
    gdp_by_state_by_inds_p2 = tabula.read_pdf(data_dir, pages = 6)

    gdp_by_state_by_inds_p2 = gdp_by_state_by_inds_p2[0]
    gdp_by_state_by_inds_p2 = gdp_by_state_by_inds_p2[2:]

    gdp_by_state_by_inds_p2[["Real estate and rental and leasing",
                      "Professional, scientific, and technical services"]] = gdp_by_state_by_inds_p2["Unnamed: 1"].str.split(" ",expand = True)

    gdp_by_state_by_inds_p2[["Educational services",
                      "Health care and social assistance"]] = gdp_by_state_by_inds_p2[
                                                            "Seasonally adjusted at annual rates"
                                                                                  ].str.split(" ", expand = True)
    gdp_by_state_by_inds_p2[["Other services (except government and government enterprises)",
                      "Government and government enterprises"]] = gdp_by_state_by_inds_p2["Unnamed: 14"].str.split(" ", expand = True)
    gdp_by_state_by_inds_p2 = gdp_by_state_by_inds_p2.drop(
                        columns= ["Unnamed: 1","Unnamed: 14","Seasonally adjusted at annual rates"])
    gdp_by_state_by_inds_p2 = gdp_by_state_by_inds_p2.dropna(axis=1)
    name_dic = {"Unnamed: 0":"areas",
                "Unnamed: 2":"Management of companies and enterprises",
                "Unnamed: 5":"Administrative and support and waste management and remediation services",
                "Unnamed: 9":"Arts, entertainment, and recreation", 
                "Unnamed: 11":"Accomodation and food services"}
    gdp_by_state_by_inds_p2 = gdp_by_state_by_inds_p2.rename(columns=name_dic)
    gdp_by_state_by_inds_p2 = gdp_by_state_by_inds_p2.reset_index(drop=True)
    # JOIN
    gdp_by_state_by_inds = pd.concat([gdp_by_state_by_inds_p1,gdp_by_state_by_inds_p2], axis = 1,)
    gdp_by_state_by_inds.columns = gdp_by_state_by_inds.columns.str.replace(
                                                "\)|\(|,| ","_").str.replace("__","_").str.lower()
    gdp_by_state_by_inds = gdp_by_state_by_inds.iloc[:,~gdp_by_state_by_inds.columns.duplicated()]
    return gdp_by_state_by_inds

In [431]:
data_dir = os.path.join(os.getcwd(),"data/qgdpstate0120_2.pdf")              

In [432]:
get_gdp_pct_by_states_by_inds(data_dir).head()

,areas,states_overall,agriculture_forestry_fishing_insuranceand_hunting,mining_quarrying_and_oil_and_gas_extraction,utilities,construction,retail_trade,transportation_and_warehousing,durable_goods_manufacturing,nondurable_goods_manufacturing,...,management_of_companies_and_enterprises,administrative_and_support_and_waste_management_and_remediation_services,arts_entertainment_and_recreation,accomodation_and_food_services,real_estate_and_rental_and_leasing,professional_scientific_and_technical_services,educational_services,health_care_and_social_assistance,other_services_except_government_and_government_enterprises_,government_and_government_enterprises
0,United States,2.1,0.06,0.11,-0.26,-0.09,0.43,-0.03,0.01,0.46,...,0.16,0.10,0.02,0.17,0.11,0.42,0.08,0.19,0.05,0.01
1,New England,2.1,0.01,0.01,-0.24,-0.09,0.39,-0.02,0.02,0.33,...,0.20,0.12,0.04,0.17,0.09,0.56,0.23,0.21,0.05,-0.01
2,Connecticut,2.1,0.00,0.00,-0.27,-0.15,0.39,-0.01,0.02,0.71,...,0.19,0.11,0.04,0.11,0.03,0.37,0.27,0.17,0.07,-0.04
3,Maine,2.1,0.01,0.00,-0.22,0.04,0.72,0.01,-0.02,0.29,...,0.27,0.15,0.05,0.33,0.07,0.26,0.06,0.15,0.05,-0.10
4,Massachusetts,2.2,0.02,0.01,-0.21,-0.11,0.31,-0.03,0.04,0.21,...,0.20,0.10,0.03,0.16,0.13,0.72,0.24,0.22,0.05,0.00


## 2.0 Employment by states
**States Data**

- NY: https://www.labor.ny.gov/stats/2019-2021-Statewide-Short-Term-Industry-Projections.xlsx


In [225]:
# headers = {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36"}

# url="https://labor.ny.gov/stats/2016-2026-Statewide-and-Regional-Long-Term-Industry-Projections.xlsx"
# s=requests.get(url, headers= headers).text

### New York

In [265]:
employment_ny = pd.read_excel(os.path.join(os.getcwd(), "data/2019-2021-Statewide-Short-Term-Industry-Projections.xlsx"))

In [266]:
employment_ny.head(10)

,Unnamed: 0,New York State Department of Labor,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,NaN,"Short-Term Industry Employment Projections, 20...",NaN,NaN,NaN,NaN,NaN
1,NaN,New York State,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NAICS\nCode,Industry Title,Employment,NaN,Net\nChange,Percent\nChange
4,NaN,NaN,NaN,2019,2021,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,000000,Total All Industries,10234640,10435820,201180,0.0196568
7,NaN,NaN,NaN,,,NaN,NaN
8,NaN,00601,"Total Self Employed, All Jobs",570590,577460,6870,0.0120402
9,NaN,NaN,NaN,,,NaN,NaN


In [267]:
employment_ny = employment_ny.dropna(axis=1,how='all')

In [268]:
employment_ny

,New York State Department of Labor,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,"Short-Term Industry Employment Projections, 20...",NaN,NaN,NaN,NaN,NaN
1,New York State,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NAICS\nCode,Industry Title,Employment,NaN,Net\nChange,Percent\nChange
4,NaN,NaN,2019,2021,NaN,NaN
...,...,...,...,...,...,...
149,NaN,State Office Campus Bldg. 12,NaN,NaN,NaN,NaN
150,NaN,Room 490,NaN,NaN,NaN,NaN
151,NaN,"Albany, NY 12240",NaN,NaN,NaN,NaN
152,NaN,Phone: (518) 457-3800,NaN,NaN,NaN,NaN


In [272]:
employment_ny = employment_ny.dropna(axis=0, how="any").reset_index(drop = True)

In [274]:
employment_ny.shape

(109, 6)

In [275]:
employment_ny.columns = ["naics_code","industry","2019","2021","net_change","pct_change"]

- `00601`: Self Employed Workers, All Jobs
- `11`: Agriculture, Forestry, Fishing and Hunting
- `21`: Mining
- `22`: Utilities
- `23`: Construction
- `31`: Manufacturing
- `42`: Wholesale Trade
- `44`: Retail Trade
- `48`: Transportation and Warehousing
- `51`: Information
- `1023`: Financial Activities
- `1024`: Professional and Business Services
- `61`: Educational Services
- `62`: Health Care and Social Assistance
- `71`: Arts, Entertainment, and Recreation
- `72`: Accommodation and Food Services
- `81`: Other Services (except Government)
- `9`: Government

In [276]:
employment_ny.naics_code = employment_ny.naics_code.astype(int)
employment_ny['2019'] = employment_ny['2019'].astype(int)
employment_ny['2021'] = employment_ny['2021'].astype(int)

In [277]:
industry_codes = [601,11,21,22,23,31,42,44,48,51,1023,1024,61,62,71,72,81,9,]

In [278]:
employment_ny_inds = employment_ny[employment_ny.naics_code.isin(industry_codes)]

In [279]:
employment_ny_inds

,naics_code,industry,2016,2026,net_change,pct_change
1,601,"Total Self Employed, All Jobs",570590,577460,6870,0.0120402
2,11,"Agriculture, Forestry, Fishing and Hunting",31180,31870,690,0.0221296
8,21,Mining,4230,4550,320,0.0756501
12,22,Utilities,36700,36430,-270,-0.00735695
13,23,Construction,380940,390570,9630,0.0252796
17,31,Manufacturing,441060,439460,-1600,-0.00362762
39,42,Wholesale Trade,322100,318890,-3210,-0.00996585
43,44,Retail Trade,930340,913410,-16930,-0.0181976
56,48,Transportation and Warehousing,279600,289240,9640,0.0344778
67,51,Information,273110,276340,3230,0.0118267


### New Jersey

## 3. Covid infection data
- data is directly from John Hopkins github repo: https://github.com/CSSEGISandData/COVID-19


In [313]:
def get_covid_data(date):
    url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{}.csv".format(date)
    covid19_df = pd.read_csv(url, index_col=0)
    covid19_df = covid19_df.reset_index()
    covid19_df = covid19_df[covid19_df["Country_Region"] == "US"]
    return covid19_df

In [314]:
covid19_df = get_covid_data(date="03-28-2020")

In [316]:
covid19_df.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
0,45001.0,Abbeville,South Carolina,US,2020-03-28 23:05:37,34.223334,-82.461707,3,0,0,0,"Abbeville, South Carolina, US"
1,22001.0,Acadia,Louisiana,US,2020-03-28 23:05:37,30.295065,-92.414197,9,1,0,0,"Acadia, Louisiana, US"
2,51001.0,Accomack,Virginia,US,2020-03-28 23:05:37,37.767072,-75.632346,2,0,0,0,"Accomack, Virginia, US"
3,16001.0,Ada,Idaho,US,2020-03-28 23:05:37,43.452658,-116.241552,76,0,0,0,"Ada, Idaho, US"
4,19001.0,Adair,Iowa,US,2020-03-28 23:05:37,41.330756,-94.471059,1,0,0,0,"Adair, Iowa, US"


## 4.0 Populations by States 2019

- link to census data: 
https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/state/detail/SCPRC-EST2019-18+POP-RES.csv


In [319]:
def get_population_data():
    url = "https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/state/detail/SCPRC-EST2019-18+POP-RES.csv"
    df = pd.read_csv(url)
    return df

In [320]:
pop_2019 = get_population_data()

In [322]:
pop_2019.head()

,SUMLEV,REGION,DIVISION,STATE,NAME,POPESTIMATE2019,POPEST18PLUS2019,PCNT_POPEST18PLUS
0,10,0,0,0,United States,328239523,255200373,77.7
1,40,3,6,1,Alabama,4903185,3814879,77.8
2,40,4,9,2,Alaska,731545,551562,75.4
3,40,4,8,4,Arizona,7278717,5638481,77.5
4,40,3,7,5,Arkansas,3017804,2317649,76.8


In [324]:
pop_2019.NAME.unique()

array(['United States', 'Alabama', 'Alaska', 'Arizona', 'Arkansas',
       'California', 'Colorado', 'Connecticut', 'Delaware',
       'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
       'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
       'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming',
       'Puerto Rico Commonwealth'], dtype=object)